传统的用于bbox回归的loss包括L1 loss与L2 loss。L1 loss对于异常值较为鲁棒，但在训练过程中梯度始终保持不变，在训练后期会在局部最优点附近震荡，难以收敛到较好的局部最优解。L2 loss对异常值较敏感，且在训练前期如果预测值与标签相差较大，可能造成训练不稳定。

smooth l1 loss综合了L1 loss与L2 loss的优点，其表达式如下：<br/>
$$
smooth_{L1}(x) = 
\begin {cases}
0.5x^2, if |x|<1 \\
|x| - 0.5, otherwise
\end {cases}
$$
其函数图像如下：<br/>
<img src="1.jpg" alt="" width="50%" height="50%" align="center" />
实际用于回归中时，smooth L1的形式如下：<br/>
$$
L_{loc}(t^u, v) = \sum_{i \in \{x,y,w,h\}}{smooth_{L1}(t_i^u - v_i)}
$$
即x,y,w,h分别进行计算

L1 loss，L2 Loss,smooth L1 Loss均是将检测框的x,y.w.h分别计算loss，忽略了bbox四个点之间的相关性，而iou Loss的提出解决了此问题。<br/>
iou-based loss的形式可以统一写为：
$$
L = 1- iou + R(bbox, gt)
$$
$其中R(bbox, gt)为预测框与gt之间的惩罚项，所有的iou-based loss只是惩罚项不同$<br/>
原论文中给出的iou loss的形式为：
$$
iou_{loss}(bbox, gt) = -ln[iou(bbox, gt)]
$$
而实现时一般用：<br/>
$$
iou_{loss}(bbox,gt) = 1 - iou(bbox, gt)
$$

iou loss有两个缺点：<br/>
1)当预测框与gt不相交时，iou loss无法进行优化（iou的计算中，有一句，if left >= right or top >= bottom: return 0，因此不相交时，iou始终为常数0，反传的梯度为0）<br/>
2)iou不考虑预测框与gt的相对位置，只要两者iou相同，则iou loss也相同，如对于以下情况，iou loss是相等的：
<img src="2.jpg" alt="" width="50%" height="50%" align="center" />

giou(global iou，通用的iou)解决了上述两个问题，其形式为：
$$
giou_{loss}(bbox, gt) = 1 - giou(bbox, gt)
$$
$其中，giou的计算方式为：$<br/>
$$
giou(bbox, gt) = iou(bbox, gt) - \frac{|c - (A \bigcup B)|}{|c|}
$$
$其中C为二者的最小外接矩形，|C|为最小外接矩形面积$

$当预测框与gt不相交时，iou等于0,giou_{loss} \iff 2 - \frac{A \bigcup B}{C}，此时我们的优化目标\\是最小化giou_{loss}，这会使|C|越来越小，因此预测框与gt会越来越接近$

giou loss还有一个缺点，就是当gt包含预测框时，iou的值与giou的值一样，此时giou退化为iou，这种情况下，所有的giou loss的值等于1 - iou，无法区分预测框与gt之间的相对位置

diou(distance-iou)的出现解决了该问题（注意diou与ciou的提出是在同一篇论文）,diou通过加入相对距离解决了gt包含预测框的问题,其形式为：
$$
L_{diou} = 1 - iou + R_{DIoU} = 1 - iou + \frac{\rho^2(bbox, gt)}{c^2}
$$
$其中，\rho(x)为bbox与gt中心点之间的欧氏距离，c为最小外接矩形对角线距离$<br/>
$diou loss遗传了iou loss与giou loss的一些特性：$<br/>
$1)尺度不变性：对于框的尺寸不敏感，不同尺寸的框可能得到相同的loss$<br/>
$2)与giou loss类似，可以解决bbox与gt不相交的情况$<br/>
$3)当bbox与gt完全重合时，loss值为0$<br/>
$同时，diou loss有其独特的优点：$<br/>
$1)diou loss直接衡量预测框与gt之间的距离，优化时收敛更快$<br/>
$2)可以解决两个框包含的问题$

同时，作者总结了一个完整的iou loss应该考虑以下三个维度：重叠区域、中心点距离以及长宽比，基于此作者提出了ciou(complete iou)loss，其形式如下：
$$
L_{ciou} = 1 - iou + R_{ciou} = 1 - iou +\frac{\rho^2(bbox, gt)}{c^2} + \alpha  \nu
$$
$其中，\nu = \frac{4}{\pi^2}(arctan\frac{w^{gt}}{h^{gt}} - arctan\frac{w^{bbox}}{h^{bbox}})^2，用于衡量bbox与gt之间长宽比的相似性，\\ \alpha为正的平衡系数$